# Explore feature space

## Setup

In [ ]:
# import libraries
import logging
import os
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from pqdm.processes import pqdm
from tqdm.notebook import tqdm

import utils

logging.basicConfig(level=logging.INFO)

In [ ]:
# visualise wind distribution
fig, ax = plt.subplots(figsize=(6, 4))
_ = utils.wfm_low.site.plot_wd_distribution(
    n_wd=len(utils.WD_DEFAULT), ws_bins=[0, 5, 10, 15, 20, 25, 30], ax=ax
)
fig.tight_layout()

In [ ]:
# visualise turbine power curve
fig, ax = plt.subplots(figsize=(6, 3))
ws = np.linspace(0, 30, 100)
ax.plot(ws, utils.wfm_low.windTurbines.power(ws) / 1e6)
ax.set_xlabel("Wind speed [m/s]")
ax.set_ylabel("Power [MW]")
fig.tight_layout()

## Baseline values
Summary values across all wind directions with no wake steering

In [ ]:
# run baseline simulation
sim_res_ref, Sector_frequency, P = utils.run_sim(
    wfm=utils.wfm_low,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=0,
    ws=utils.WS_DEFAULT,
    wd=utils.WD_DEFAULT,
    show=True,
)

## Assess across coarse range for all turbines
Assess impact of wake steering a fixed yaw angle in either direction for all turbines to determine which turbines and directions have an impact

In [ ]:
coarse_dir = os.path.join("data", "coarse_sweep.h5")
yaw_ind_coarse = {f"yaw_{i}": [-5, 0, 5] for i in sim_res_ref.wt.values}
if os.path.isfile(coarse_dir):
    # load pre-existing dataset
    ds_coarse = xr.open_dataset(coarse_dir)
else:
    # initialise dataset
    dims_all = ["wt", "wd"] + list(yaw_ind_coarse.keys())
    shape_all = (len(sim_res_ref.wt), len(sim_res_ref.wd)) + tuple(
        len(x) for x in yaw_ind_coarse.values()
    )
    dims_direction = dims_all[1:]
    shape_direction = shape_all[1:]
    dims_overall = dims_direction[1:]
    shape_overall = shape_direction[1:]
    ds_coarse = xr.Dataset(
        data_vars={
            "aep": (dims_all, np.full(shape_all, np.nan)),
            "lcoe": (dims_all, np.full(shape_all, np.nan)),
            "cap_fac": (dims_all, np.full(shape_all, np.nan)),
            "lcoe_direction": (dims_direction, np.full(shape_direction, np.nan)),
            "cap_fac_direction": (dims_direction, np.full(shape_direction, np.nan)),
            "lcoe_overall": (dims_overall, np.full(shape_overall, np.nan)),
            "cap_fac_overall": (dims_overall, np.full(shape_overall, np.nan)),
        },
        coords={"wt": list(sim_res_ref.wt.values), "wd": list(sim_res_ref.wd.values)}
        | yaw_ind_coarse,
    )
    # calculate across full range of options
    yaw_options_coarse = list(product(*yaw_ind_coarse.values()))
    simulations = {
        k: v
        for k, v in zip(
            yaw_options_coarse,
            pqdm(
                [
                    {
                        "wfm": utils.wfm_low,
                        "x": utils.wt9_x,
                        "y": utils.wt9_y,
                        "yaw": yaw,
                        "ws": utils.WS_DEFAULT,
                        "wd": utils.WD_DEFAULT,
                        "sim_res_ref": sim_res_ref,
                        "Sector_frequency": Sector_frequency,
                        "P": P,
                    }
                    for yaw in yaw_options_coarse
                ],
                utils.run_sim,
                n_jobs=int(0.75 * os.cpu_count()),
                argument_type="kwargs",
            ),
        )
    }
    # save results to dataset
    for yaw_option, (sim_res, _, _) in tqdm(
        simulations.items(), total=len(simulations), desc="Saving values to dataset"
    ):
        ds_coarse["aep"].loc[:, :, *yaw_option] = sim_res.energy
        ds_coarse["lcoe"].loc[:, :, *yaw_option] = sim_res.lcoe
        ds_coarse["cap_fac"].loc[:, :, *yaw_option] = sim_res.cap_fac
        ds_coarse["lcoe_direction"].loc[:, *yaw_option] = sim_res.lcoe_direction
        ds_coarse["cap_fac_direction"].loc[:, *yaw_option] = sim_res.cap_fac_direction
        ds_coarse["lcoe_overall"].loc[*yaw_option] = sim_res.lcoe_overall
        ds_coarse["cap_fac_overall"].loc[*yaw_option] = sim_res.cap_fac_overall
    # save file
    ds_coarse.to_netcdf(coarse_dir)

In [ ]:
# identify optimums for each wind direction
yaw_opt_coarse = ds_coarse.lcoe_direction.stack(
    dimensions={"all_dims": yaw_ind_coarse.keys()}
).idxmin(dim="all_dims")
yaw_opt_coarse.to_dataframe().drop("all_dims", axis=1)

In [ ]:
# plot improvements
fig, ax_all = plt.subplots(ncols=2, figsize=(8, 3))
(
    ds_coarse.lcoe_direction.min(list(yaw_ind_coarse.keys()))
    - ds_coarse.lcoe_direction.sel({k: 0 for k in yaw_ind_coarse})
).to_series().plot.bar(ax=ax_all[0])
ax_all[0].set_ylabel("lcoe_improvement")
(
    100
    * (
        ds_coarse.cap_fac_direction.max(list(yaw_ind_coarse.keys()))
        - ds_coarse.cap_fac_direction.sel({k: 0 for k in yaw_ind_coarse})
    )
).to_series().plot.bar(ax=ax_all[1])
ax_all[1].set_ylabel("cap_fac_improvement")
fig.tight_layout()

In [ ]:
# plot flow maps
fig, ax_all = plt.subplots(ncols=4, nrows=6, figsize=(15, 15))
for wd, ax in tqdm(zip(ds_coarse.wd.values, ax_all.flatten()), total=len(ds_coarse.wd)):
    sim_res, _, _ = utils.run_sim(
        wfm=utils.wfm_low,
        x=utils.wt9_x,
        y=utils.wt9_y,
        yaw=yaw_opt_coarse.sel(wd=wd).values.tolist(),
        ws=12,
        wd=[wd],
        Sector_frequency=Sector_frequency,
        P=P,
    )
    fm = sim_res.flow_map()
    fm.plot_wake_map(ax=ax)
    ax.grid()
    ax.set_title(f"wd={wd}")
fig.tight_layout()

Observations
- Improvements observied in every direction but direction with existing offset and downstream turbines near to wake benefit most as they require only a small deviation
- Movement of last front row is pointless
- Middle rows don't seem to benefit from changing (in this coarse system anyway) with no pre-existing offset

## Assess across fine range of yaw combinations for two turbines
Assess impact of wake steering a range of yaw angles in either direction for two turbines to determine how much the front and middle row need to yaw for optimality

In [ ]:
fine_dir = os.path.join("data", "fine_sweep.h5")
yaw_ind_fine = {
    f"yaw_{i}": np.arange(-30, 30.5, 0.5) for i in sim_res_ref.wt.values[[1, 4]]
}
if os.path.isfile(fine_dir):
    # load pre-existing dataset
    ds_fine = xr.open_dataset(fine_dir)
else:
    # initialise dataset
    dims_all = ["wt", "wd"] + list(yaw_ind_fine.keys())
    shape_all = (len(sim_res_ref.wt), len(sim_res_ref.wd)) + tuple(
        len(x) for x in yaw_ind_fine.values()
    )
    dims_direction = dims_all[1:]
    shape_direction = shape_all[1:]
    dims_overall = dims_direction[1:]
    shape_overall = shape_direction[1:]
    ds_fine = xr.Dataset(
        data_vars={
            "aep": (dims_all, np.full(shape_all, np.nan)),
            "lcoe": (dims_all, np.full(shape_all, np.nan)),
            "cap_fac": (dims_all, np.full(shape_all, np.nan)),
            "lcoe_direction": (dims_direction, np.full(shape_direction, np.nan)),
            "cap_fac_direction": (dims_direction, np.full(shape_direction, np.nan)),
            "lcoe_overall": (dims_overall, np.full(shape_overall, np.nan)),
            "cap_fac_overall": (dims_overall, np.full(shape_overall, np.nan)),
        },
        coords={"wt": list(sim_res_ref.wt.values), "wd": list(sim_res_ref.wd.values)}
        | yaw_ind_fine,
    )
    # calculate across full range of options
    yaw_options_fine = list(
        product(*[yaw_ind_fine.get(f"yaw_{x}", [0]) for x in sim_res_ref.wt.values])
    )
    simulations = {
        k: v
        for k, v in zip(
            yaw_options_fine,
            pqdm(
                [
                    {
                        "wfm": utils.wfm_low,
                        "x": utils.wt9_x,
                        "y": utils.wt9_y,
                        "yaw": yaw,
                        "ws": utils.WS_DEFAULT,
                        "wd": utils.WD_DEFAULT,
                        "sim_res_ref": sim_res_ref,
                        "Sector_frequency": Sector_frequency,
                        "P": P,
                    }
                    for yaw in yaw_options_fine
                ],
                utils.run_sim,
                n_jobs=int(0.75 * os.cpu_count()),
                argument_type="kwargs",
            ),
        )
    }
    # save results to dataset
    for yaw_option, (sim_res, _, _) in tqdm(
        simulations.items(), total=len(simulations), desc="Saving values to dataset"
    ):
        ds_fine["aep"].loc[:, :, yaw_option[1], yaw_option[4]] = sim_res.energy
        ds_fine["lcoe"].loc[:, :, yaw_option[1], yaw_option[4]] = sim_res.lcoe
        ds_fine["cap_fac"].loc[:, :, yaw_option[1], yaw_option[4]] = sim_res.cap_fac
        ds_fine["lcoe_direction"].loc[
            :, yaw_option[1], yaw_option[4]
        ] = sim_res.lcoe_direction
        ds_fine["cap_fac_direction"].loc[
            :, yaw_option[1], yaw_option[4]
        ] = sim_res.cap_fac_direction
        ds_fine["lcoe_overall"].loc[yaw_option[1], yaw_option[4]] = sim_res.lcoe_overall
        ds_fine["cap_fac_overall"].loc[
            yaw_option[1], yaw_option[4]
        ] = sim_res.cap_fac_overall
    # save file
    ds_fine.to_netcdf(fine_dir)

In [ ]:
# identify optimums for each wind direction

# calculate optimum for each direction
yaw_opt_fine = ds_fine.lcoe_direction.stack(
    dimensions={"all_dims": yaw_ind_fine.keys()}
).idxmin(dim="all_dims")
# fill out 0 values for unoptimised turbines
data_list = []
for vals in yaw_opt_fine.values:
    yaw_opt = list((0,) * len(ds_fine.wt))
    yaw_opt[1], yaw_opt[4] = vals
    data_list.append(tuple(yaw_opt))
data = np.empty(len(data_list), dtype=object)
data[:] = data_list
# store as dataarray
yaw_opt_fine = xr.DataArray(data=data, coords={"wd": yaw_opt_fine.wd.values})

In [ ]:
# plot improvements
fig, ax_all = plt.subplots(ncols=2, figsize=(8, 3))
(
    ds_fine.lcoe_direction.min(list(yaw_ind_fine.keys()))
    - ds_fine.lcoe_direction.sel(yaw_1=0, yaw_4=0)
).to_series().plot.bar(ax=ax_all[0])
ax_all[0].set_ylabel("lcoe_improvement")
(
    100
    * (
        ds_fine.cap_fac_direction.max(list(yaw_ind_fine.keys()))
        - ds_fine.cap_fac_direction.sel(yaw_1=0, yaw_4=0)
    )
).to_series().plot.bar(ax=ax_all[1])
ax_all[1].set_ylabel("cap_fac_improvement")
fig.tight_layout()

In [ ]:
# plot distribution of lcoe values
fig, ax_all = plt.subplots(ncols=4, nrows=6, figsize=(15, 15))
for wd, ax in zip(ds_fine.wd, ax_all.flatten()):
    _ = ds_fine.lcoe_direction.sel(wd=wd.values).plot.contourf(levels=25, ax=ax)
fig.tight_layout()

In [ ]:
# plot distribution of cap_fac values
fig, ax_all = plt.subplots(ncols=4, nrows=6, figsize=(15, 15))
for wd, ax in zip(ds_fine.wd, ax_all.flatten()):
    _ = (100 * ds_fine.cap_fac_direction.sel(wd=wd.values)).plot.contourf(
        levels=25, ax=ax
    )
fig.tight_layout()

In [ ]:
# plot flow maps
fig, ax_all = plt.subplots(ncols=4, nrows=6, figsize=(15, 15))
for wd, ax in zip(ds_fine.wd.values, ax_all.flatten()):
    sim_res, _, _ = utils.run_sim(
        wfm=utils.wfm_low,
        x=utils.wt9_x,
        y=utils.wt9_y,
        yaw=yaw_opt_fine.sel(wd=wd).values.tolist(),
        ws=12,
        wd=[wd],
        Sector_frequency=Sector_frequency,
        P=P,
    )
    fm = sim_res.flow_map()
    fm.plot_wake_map(ax=ax)
    ax.grid()
    ax.set_title(f"wd={wd}")
fig.tight_layout()

In [ ]:
# plot distribution of wind speed and power changes
wd = 0
print(list(yaw_opt_fine.sel(wd=wd).values.tolist())[3:6])
sim_res_opt_fine, _, _ = utils.run_sim(
    wfm=utils.wfm_low,
    x=utils.wt9_x,
    y=utils.wt9_y,
    yaw=yaw_opt_fine.sel(wd=wd).values.tolist(),
    ws=utils.WS_DEFAULT,
    wd=utils.WD_DEFAULT,
)
fig, ax = plt.subplots(ncols=3, figsize=(15, 2.5))
WS_eff_diff = (sim_res_opt_fine.WS_eff - sim_res_ref.WS_eff).sel(
    {"wt": [3, 4, 5], "wd": wd}
)
Power_diff = (sim_res_opt_fine.Power - sim_res_ref.Power).sel(
    {"wt": [3, 4, 5], "wd": wd}
)
WS_eff_diff.plot(ax=ax[0])
Power_diff.plot(ax=ax[1])
Power_diff.sum("wt").plot(ax=ax[2])
ax[0].set_yticks(WS_eff_diff.wt)
ax[1].set_yticks(Power_diff.wt)
fig.tight_layout()

Observations
- Rare for front and middle rows to be moved
- Direction of movement favours prexisting offset if one exists
- Considerable improvements observed even with diagonal wind directions which have better spacing between wakes
- Off design performance is good - not too sensitive near optimum
- Improvements increase with wind speed until maximal power output
    - Near maximal power output speed the increase of the second turbine is minimal as the power curve flattens so a net loss is observed
- Optimal yaw likely a function of wind speed and direction
    - Change with speed appears smoother than changes with direction